<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
import time
import os
import random
import numpy as np

In [9]:
# Create a dictionary to save memory later
move = {None: "Initial Block", 0 : "NONE", -1 : "UP", 1 : "DOWN", -2 : "LEFT", 2 : "RIGHT"}
state = {0 : "STANDING", 1 : "LYING_HORIZONTALLY", 2 : "LYING_VERTICALLY", 3 : "SPLIT"}
square = {0 : "NONE", 1 : "WEAK", 2 : "STRONG", 3 : "WEAK_SWICH", 4 : "STRONG_SWITCH", 5 : "TELEPORT"}
on = {0 : "OFF", 1 : "ON"}

In [51]:
# Position of the block in the map
class Block:
  def __init__(self, parts, last_move = None):
    self.parts = sorted(parts, key = lambda x : 1000*x[0] + x[1])
    self.state = self.get_state()
    self.last_move = last_move

  def get_state(self):
    row_diff = abs(self.parts[0][0] - self.parts[1][0])
    col_diff = abs(self.parts[0][1] - self.parts[1][1])

    if row_diff == 0 and col_diff == 0: return 0
    elif row_diff == 0 and col_diff == 1: return 1
    elif row_diff == 1 and col_diff == 0: return 2
    else: return 3

  # def is_valid(self):
  #   return self.parts[0][0] >= 0 and self.parts[0][1] >= 0 and self.parts[1][0] >= 0 and self.parts[1][1] >= 0
  
  def move(self, instruction, movable_part = None):
    if instruction == 0: return copy.copy(self)

    child_parts = copy.deepcopy(self.parts)
    # Standing
    if self.state == 0:
      if instruction == -1:
        child_parts[0][0] -= 2
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 2
      elif instruction == -2:
        child_parts[0][1] -= 2
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 2
    # Lying horizontally
    elif self.state == 1:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 2
      else:
        child_parts[0][1] += 2
        child_parts[1][1] += 1
    # Lying vertically
    elif self.state == 2:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 2
      elif instruction == 1:
        child_parts[0][0] += 2
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 1
    #Split
    else: child_parts[movable_part][abs(instruction) - 1] += np.sign(instruction)

    if self.state == 3: return Block(child_parts, (instruction, movable_part))
    else: return Block(child_parts, instruction)
    

In [23]:
class Special_square:
  def __init__ (self, name, position, affects, on = 0):
    self.name = name
    self.postion = position
    self.affects = affects
    self.on = on

  def change(self):
    self.on = 1 - self.on
  
  def print(self):
    # Weak switch
    if self.name == 3: 
      if self.on == 1: print("Weak switch at", self.postion, "is now on, squares", self.affects, "are added!")
      else: print("Weak switch at", self.postion, "is now off, squares", self.affects, "are removed!")
    if self.name == 4:
      if self.on == 1: print("Strong switch at", self.postion, "is now on, squares", self.affects, "are added!")
      else: print("Strong switch at", self.postion, "is now off, squares", self.affects, "are removed!")
    if self.name == 5: print("Teleport at", self.postion, "splits blocks into", self.affects[0], "and", self.affects[1])

In [52]:
class State:
  def __init__ (self, map, block, specials = None, parent = None):
    self.map = map
    self.block = block
    self.specials = specials
    self.parent = parent
    
    self.update()

  # Get value of current block's position in the map
  def get_value(self, part):
    return map[self.block.parts[0][0]][self.block.parts[0][1]]

  def is_valid(self):
    if self.block.parts[0][0] not in range(len(map)) or self.block.parts[1][0] not in range(len(map)): return False
    if self.block.parts[0][1] not in range(len(map[0])) or self.block.parts[1][1] not in range(len(map[0])): return False
    if self.block.state == 0 and self.get_value(0) < 2: return False
    if self.get_value(0) < 0 or self.get_value(1) < 0: return False

    return True
  
  def find_special(self, position):
    return filter(lambda x: x.position == position, self.specials)

  #Check if there is a special squared changes its state and return it
  def special_change(self):
    result = None
    # At least one parts in on weak switch
    if self.get_value(0) == 3 and self.parent.get_value(0) != 3:
      result = self.find_special(self.block.parts[0])
    if self.get_value(1) == 3 and self.parent.get_value(1) != 3:
      result = self.find_special(self.block.parts[1])
    
    # Block is standing on strong switch
    if self.block.state == 0 and self.get_value(0) == 4 and self.parent.get_value(0) != 4:
      result = self.find_special(self.block.parts[0])
    
    # Teleport
    if self.get_value(0) == 5:
      result = self.find_special(self.block.parts[0])
    if self.get_value(1) == 5:
      result = self.find_special(self.block.parts[1])


  def update(self):
    change = self.special_change()
    if change == None: return

    # At least one parts in on weak switch
    if self.get_value(0) == 3 or self.get_value(0) == 3:
      weak_switch = None
      if self.get_value(0) == 3 and self.parent.get_value(0) != 3:
        weak_switch = self.find_special(self.block.parts[0])
      if self.get_value(1) == 3 and self.parent.get_value(1) != 3:
        weak_switch = self.find_special(self.block.parts[1])
      
      weak_switch.change()
      for position in weak_switch.affects:
        if weak_switch.on == 1: map[position[0]][position[1]] = 2
        else: self.map[position[0]][position[1]] = 0
    
    elif :
      strong_switch = None
      if self.get_value(0) == 4 and :
        strong_switch = self.find_special(self.block.parts[0])
      
      strong_switch.change()
      for position in strong_switch.affects:
        if strong_switch.on == 1: map[position[0]][position[1]] = 2
        else: self.map[position[0]][position[1]] = 0
    # Teleport
    elif 
      teleport = self.find_special(self.block.parts[0])
      self.block.parts = Block(teleport.affects)

  def move(self, instruction, movable_part = None):
    child_block = self.block.move(instruction, movable_part)
    return State(copy.deepcopy(self.map), child_block, copy.deepcopy(self.specials), self)

  def expand(self):
    children = []
    if self.block.state == 3:
      for i in [-1, 1, -2, 2]:
        children.append(self.move(i, 0))
        children.append(self.move(i, 1))
    else:
      for i in [-1, 1, -2, 2]:
        children.append(self.move(i))
    return children

In [58]:
level1 = State([[2,2,2,0,0,0,0,0,0,0],
                  [2,2,2,2,2,2,0,0,0,0],
                  [2,2,2,2,2,2,2,2,2,0],
                  [0,2,2,2,2,2,2,2,2,2], 
                  [0,0,0,0,0,2,2,10,2,2], 
                  [0,0,0,0,0,0,2,2,2,0]], Block([[1,1],[3,4]]))
children = level1.expand()

for child in children: print(child.block.parts)

[[0, 1], [3, 4]]
[[1, 1], [2, 4]]
[[2, 1], [3, 4]]
[[1, 1], [4, 4]]
[[1, 0], [3, 4]]
[[1, 1], [3, 3]]
[[1, 2], [3, 4]]
[[1, 1], [3, 5]]


In [25]:
a = [1, 2, 3]
a.append(None)

print(a)

[1, 2, 3, None]


In [ ]:
 # This is the original DFS
def helper(game, stack, visited, loop):
  while len(stack) > 0:
    game.current = stack.pop()
    visited.append(game.current.parts)
    children = game.expand() 
    # if game.current.parent != None and game.current.parent.parts == [[0, 9], [1, 9]]: 
    # if game.current.parent != None:
    #   print(game.current.parent.parts, move[game.current.move_from_parent], game.current.parts)
    for child in children:
      
      if game.is_goal(child): 
        print("Goal:", child.parts)
        return child, loop
      if child.parts not in visited: stack.append(child)

    loop = loop + 1
  return -1, loop

#This is used to call DFS more conveniently
def DFS(game):
  index = 0;
  goal, loop = helper(game, [game.initial_block], [], 0)
  if goal == -1: print("No solutions")
  else:
    path = [goal]
    while path[-1].parent != None: path.append(path[-1].parent)
    path.reverse()
    print("Goal reached after", loop, "loops")
    for position in path: 
      print("Steps", index, move[position.move_from_parent],":", position.parts)
      index = index + 1

In [ ]:
class BLOXORZ:
  def __init__(self, map, initial_block, current = None):
    self.map = map
    self.initial_block = initial_block
    self.goal = [(index, row.index(10)) for index, row in enumerate(map) if 10 in row][0]
    self.current = initial_block

  # Get value at part_number of current block
  def get_value(self, part_number):
    return self.map[self.current.parts[part_number][0]][self.current.parts[part_number][1]]

  def is_goal(self, current = None):
    if current == None: current = self.state
    return current.state == 0 and self.map[current.parts[0][0]][current.parts[0][1]] == 10

  # Check if current block is valid in this map
  def is_valid(self, current = None):
    if current == None: current = self.current

    if current.parts[0][0] < 0 or current.parts[1][0] < 0 or current.parts[0][1] < 0 or current.parts[1][1] < 0: 
      return False
    if current.parts[0][0] >= len(self.map) or current.parts[1][0] >= len(self.map) or current.parts[0][1] >= len(self.map[0]) or current.parts[1][1] >= len(self.map[0]): 
      return False

    if current.state == 0: 
      return self.map[current.parts[0][0]][current.parts[0][1]] > 1
    else: 
      return self.map[current.parts[0][0]][current.parts[0][1]] > 0 and self.map[current.parts[1][0]][current.parts[1][1]] > 0

  def expand(self, current = None):
    if current == None: current = self.current
    children = []
    for instruction in [-1,1,-2,2]:
      child = current.move(instruction)
      if self.is_valid(child): children.append(child)
    return children
  
  # Estimate cost to goal, calculated my average of Manhattan distances of 2 part of the block or Chebychev distance
  def cost_to_goal(self, current):
    #Average Manhattan
    average = 1/2 * (abs(current.parts[0][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[1]) + abs(current.parts[1][1] - self.goal[1]))
    return 1/average
    # Chebychev
    # max1 = max(abs(current.parts[0][0] - self.goal[0]), abs(current.parts[0][1] - self.goal[1]))
    # max2 = max(abs(current.parts[1][0] - self.goal[0]), abs(current.parts[1][1] - self.goal[1]))
    # return 1/max(max1, max2)

  def move_process(self, process):
    current = copy.copy(self.initial_block)
    for move in process: 
      current = current.move(move)
      if not self.is_valid(current): 
        return -1
    return current


In [ ]:
level1 = BLOXORZ([[2,2,2,0,0,0,0,0,0,0],
                  [2,2,2,2,2,2,0,0,0,0],
                  [2,2,2,2,2,2,2,2,2,0],
                  [0,2,2,2,2,2,2,2,2,2], 
                  [0,0,0,0,0,2,2,10,2,2], 
                  [0,0,0,0,0,0,2,2,2,0]], (1,1,1,1))

level2 = BLOXORZ()

In [ ]:
class DFS:
  # This is the original DFS
  def helper(self, game, stack, visited, loop):
    while len(stack) > 0:
      this_position = stack.pop()
      visited.append([this_position.i1, this_position.j1, this_position.i2, this_position.j2])
      exp_list = this_position.expand()
      for pos in exp_list:
          if game.is_valid_position(pos):
            if not pos.is_visited(visited):
              stack.append(pos)
              # self.draw_map_2D(pos)
            if game.is_goal(pos): return pos, loop    
      loop = loop + 1
    return -1
  
  #This is used to call DFS more conveniently
  def go(self, game):
    index = 0;
    goal, loop = self.helper(game, [game.initial_position], [], 0)
    if goal == -1: print("No solutions")
    else:
      path = [goal]
      while path[-1].parent != None: path.append(path[-1].parent)
      path.reverse()
      print("Goal reached after", loop, "loops")
      print("Steps: ")
      for position in path: 
        print(index,": (", position.i1, position.i2, ")", "(", position.j1, position.j2, ")")
        index = index + 1

In [ ]:
class Genetic():
  population = []
  prob = []
  def __init__(self, game, chromosome_length, population_capacity):
    self.game = game
    self.chromosome_length = chromosome_length
    self.population_capacity = population_capacity
    self.initPopulation()
    self.probCal()

  def initIndividual(self):
    chromosome = []
    for i in range(self.chromosome_length):
      # if chromosome is empty or having NONE as the last gen, add an arbitrary
      if len(chromosome) == 0 or chromosome[-1] == 0: chromosome.append(random.choice((0,1,2,3,4)))
      # if the last gen is UP, avoid DOWN
      elif chromosome[-1] == 1: chromosome.append(random.choice((0,1,3,4)))
      # if the last gen is DOWN, avoid UP
      elif chromosome[-1] == 2: chromosome.append(random.choice((0,2,3,4)))
      # if the last gen is LEFT, avoid RIGHT
      elif chromosome[-1] == 3: chromosome.append(random.choice((0,1,2,3)))
      # if the last gen is LEFT, avoid RIGHT
      else: chromosome.append(random.choice((0,1,2,4)))
    return chromosome
  
  def initPopulation(self):
    while len(self.population) < self.population_capacity:
      individual = self.initIndividual()
      final = self.game.move_process(individual)
      if final != -1: self.population.append(individual)

  def probCal(self):
    for individual in self.population:
      final = self.game.move_process(individual)
      self.prob.append(self.game.cost_to_goal(final))
    _sum = sum(self.prob)
    for i in range(len(self.prob)): self.prob[i] = self.prob[i] / _sum;

  #Seclect a parent in population
  def select(self):
    rannum = random.uniform(0, 1)
    for i in range (self.population_capacity):
      rannum = rannum - self.prob[i]
      if rannum < 0: return self.population[i-1]

  def cross_over(self):
    parent1 = self.select()
    parent2 = self.select()

    ran_index = random.ranint(0, self.chromosome_length)

    for i in range(ran_index, self.chromosome_length): parent1[i] = parent2[i]

    return parent1

  def mutate(self, chromosome):
     ran_index = random.ranint(0, self.chromosome_length)
     chromosome[ran_index] = random.choice((0,1,2,3,4))

  def populationUpdate(self):
    next_population = []

    # Cross over
    while (len(next_population) < self.population_capacity):
      child = self.cross_over()
      if self.game.move_process(child) != -1: next_population.append(child)
    # It is better no mutation
    
    for child in next_population: self.mutate(child)

    self.population = next_population
    self.probCal

  # Loop for 100 times
  def go(self):
    for i in range(100):
      for individual in self.population:
        if self.game.move_process(individual).is_goal(): return individual
      self.populationUpdate()

    return "Cannot find Solution"

In [ ]:
initBlock = Position(3,1,3,1)
map = [[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
        [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
        [2,2,2,2,2,2,2,2,2,0,0,2,2,2,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,10,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
        [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]]
game = BLOXORZ(map, initBlock)

# dfs = DFS()

# dfs.go(game)

# game.move_process([1,2,0,3,2])

# for move in [1,2,0,3,2]:
#   initBlock = initBlock.move(move)
#   initBlock.print()

GA = Genetic(game, 5, 10)
print(GA.population)
print(GA.prob)



[[1, 4, 2, 3, 2], [0, 4, 2, 0, 3], [0, 4, 1, 1, 3], [4, 1, 4, 0, 0], [1, 0, 4, 0, 2], [0, 1, 0, 0, 4], [4, 0, 2, 3, 1], [4, 0, 0, 2, 0], [4, 1, 0, 0, 4], [4, 2, 0, 1, 0]]
[0.08658273047025031, 0.08991283548833687, 0.08349049009631279, 0.11688668613483792, 0.10626062375894356, 0.09350934890787033, 0.09350934890787033, 0.10164059663898949, 0.11688668613483792, 0.1113206534617504]
